# Datos Colombia

En este Notebook se implementa un scraper para Datos Colombia. Esta página se vuelve más complicada debido a que tiene validaciones contra búsquedas automatizadas.

Este código permitirá realizar el almacenamiento de la información y la guardará en Google Drive para evitar temas de desconexión por runtime.

La página cuenta con 3137 vistas con 13 colegios, lo que permitiría extraer información de 40781 instituciones educativas en todo el país.

In [ ]:
from urllib.parse import unquote
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import requests
import copy
import time
import os
import random

In [ ]:
def extract_body_for_urls(idx):
  """
  Permite extraer las URLs de los colegios que se encuentran listados en la página.

  Params
  --------
    idx (int):
      Indicador del número de página. De 1 a 3137.


  Returns
  --------

    schools (dict):
      Un diccionario con el nombre de cada colegio de la página y su url
      {name:url}

  """
  index_url = f'https://datoscolombia.com/escuelas-colegios/page/{idx}'
  base_url = 'https://datoscolombia.com'

  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

  # Se debe hacer el request como si fuera un agente, no automático.
  response = requests.get(index_url, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')

  # La página contiene links de cada colegio
  school_links = soup.find_all('a', href=lambda href: href and href.startswith('/escuelas-colegios/'))
  schools = {}
  for link in school_links:
      href = base_url + link.get('href')
      name = link.get_text()
      schools[name] = href

  return schools


In [ ]:
def extract_school_information(school_url):
  """
  Extrae la información de cada colegio según lo disponible en la página.

  Params
  --------
    school_url (str):
      La url del colegio del que se extraerá la información

  Returns
  --------
    school_info (dict):
      Un diccionario que tiene toda la información extraída

  """
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
  response = requests.get(school_url, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  info_divs = soup.find_all('div', class_='info-est')

  # Diccionario de Información Extraída
  school_info = {}

  for div in info_divs:
      info_fields = div.find_all('p')

      for field in info_fields:
          label = field.find('label', class_='title-field')
          if label:
              label_text = label.get_text(strip=True)
              field_copy = copy.copy(field)
              for l in field_copy.find_all('label', class_='title-field'):
                  l.decompose()

              value_text = field_copy.get_text(strip=True)
              school_info[label_text] = value_text

  return school_info

In [ ]:
output_file = 'drive/MyDrive/datos_colombia_information.csv'

In [ ]:

if os.path.isfile(output_file):
  df = pd.read_csv(output_file)
  max_pages = df.page_num.max()
  column_order = df.columns.values.tolist()
else:
  max_pages = 1
  column_order = []

In [ ]:


for idx in tqdm(range(max_pages, 3138)):
  school_urls = extract_body_for_urls(idx)
  time.sleep(random.uniform(1, 10))

  # Lista para almacenar la información
  complete_school_extraction = []

  for school_name, school_url in school_urls.items():
    time.sleep(1)
    school_info = extract_school_information(school_url)
    school_info['url'] = school_url
    school_info['name'] = school_name
    school_info['page_num'] = idx
    time.sleep(random.uniform(1, 5))
    complete_school_extraction.append(school_info)

    # Estructuración en Data Frame
    df = pd.DataFrame(complete_school_extraction)

    new_columns = [c for c in df.columns if c not in column_order]
    column_order.extend(new_columns)

    df = df.reindex(columns=column_order)



  # Append si existe, y creación en primera iteración
  if os.path.isfile(output_file):
    df.to_csv(output_file, mode='a', header=False, index=False)
  else:
    df.to_csv(output_file, mode='w', header=True, index=False)

  2%|▏         | 34/1439 [59:17<40:30:30, 103.79s/it]